In [2]:
from ultralytics import YOLO

model = YOLO('best.pt')  # initialize

model.export(format='onnx')  # export to onnx

Ultralytics YOLOv8.1.27 🚀 Python-3.11.6 torch-2.2.1+cpu CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
YOLOv8n-cls summary (fused): 73 layers, 1441285 parameters, 0 gradients, 3.3 GFLOPs

PyTorch: starting from 'best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5) (2.8 MB)
requirements: Ultralytics requirement ['onnx>=1.12.0'] not found, attempting AutoUpdate...
   ---------------------------------------- 14.3/14.3 MB 1.2 MB/s eta 0:00:009

requirements: AutoUpdate success ✅ 74.2s, installed 1 package: ['onnx>=1.12.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: export success ✅ 78.3s, saved as 'best.onnx' (5.5 MB)

Export complete (82.3s)
Results saved to C:\Data\Software Engineer 2024\Semester VIII (Tugas Akhir)\Machine-Learning
Predict:         yolo predict task=classify model=best.onnx imgsz=640  
Validate:        yolo val task=classify model=best.onnx imgsz=640 data=/

'best.onnx'

In [9]:
from ultralytics import YOLO
model = YOLO('best.pt')  # initialize

result = model('plastik-kresek.jpg')  # 

for res in result:
    max_prob_index = res.probs.data.numpy().argmax()
    # Map the index to the class name
    max_prob_class = res.names[max_prob_index]
    # Print the class with the maximum probability



image 1/1 c:\Data\Software Engineer 2024\Semester VIII (Tugas Akhir)\Machine-Learning\plastik-kresek.jpg: 640x640 LDPE 1.00, HDPE 0.00, PET 0.00, PS 0.00, PP 0.00, 64.4ms
Speed: 15.2ms preprocess, 64.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
[ultralytics.engine.results.Results object with attributes:

boxes: None
keypoints: None
masks: None
names: {0: 'HDPE', 1: 'LDPE', 2: 'PET', 3: 'PP', 4: 'PS'}
obb: None
orig_img: array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255,

In [39]:
import onnxruntime as ort
import numpy as np
import cv2

class_names = ["HDPE", "LDPE", "PET", "PP", "PS"]

plastic_dict = {
    'HDPE': {
        'kode': 'hdpe',
        'nama': 'High density polyethylene',
        'manfaat': 'manfaat HDPE (High-Density Polyethylene) sebagai bahan plastik termasuk ketahanan terhadap bahan kimia, seperti asam dan alkali, serta tahan terhadap kelembaban dan air. HDPE juga dapat didaur ulang untuk mengurangi limbah plastik dan memiliki kekuatan tarik yang tinggi, membuatnya cocok untuk aplikasi yang membutuhkan kekuatan dan keandalan.',
        'kekurangan': 'HDPE juga memiliki beberapa kekurangan. Misalnya, HDPE rentan terhadap suhu tinggi dan dapat deformasi atau meleleh jika terpapar suhu yang ekstrem. Selain itu, HDPE tidak transparan, sehingga tidak cocok untuk produk yang memerlukan transparansi atau tampilan visual. HDPE juga tidak tahan terhadap panas dan dapat retak jika terkena tekanan atau gaya bengkok yang ekstrem.',
        'logo': 'https://storage.googleapis.com/trashurebucket/dic_ml/Symbol_Resin_Code_2_HDPE.svg'
    },
    'LDPE': {
        'kode': 'ldpe',
        'nama': 'Low density polyethylene',
        'manfaat': 'LDPE (Low-Density Polyethylene) memiliki beberapa manfaat yang membuatnya populer dalam berbagai aplikasi. Pertama, LDPE memiliki sifat tahan terhadap bahan kimia, seperti asam dan alkali, menjadikannya pilihan yang baik untuk produk-produk yang memerlukan ketahanan terhadap korosi. Kedua, LDPE memiliki sifat tahan terhadap kelembaban dan air, sehingga sering digunakan dalam produk-produk yang terpapar kelembaban tinggi. Ketiga, LDPE memiliki kekuatan yang baik dan tahan terhadap tekanan, menjadikannya pilihan yang baik untuk produk-produk yang membutuhkan kekuatan mekanis yang tinggi.',
        'kekurangan': 'LDPE juga memiliki beberapa kekurangan yang perlu dipertimbangkan. Pertama, LDPE rentan terhadap suhu tinggi dan dapat meleleh atau deformasi jika terpapar suhu yang ekstrem. Kedua, LDPE memiliki ketahanan rendah terhadap sinar UV dan dapat mengalami degradasi jika terpapar langsung sinar matahari. Ketiga, LDPE sulit untuk didaur ulang secara efisien dan membutuhkan proses khusus untuk daur ulang yang efektif.',
        'logo': 'https://storage.googleapis.com/trashurebucket/dic_ml/Symbol_Resin_Code_4_LDPE.svg'
    },
    'PET': {
        'kode': 'pet',
        'nama': 'Polyethylene terephthalate',
        'manfaat': 'PET (Polyethylene Terephthalate) memiliki beberapa manfaat yang membuatnya populer dalam berbagai aplikasi. Pertama, PET memiliki sifat tahan terhadap bahan kimia, seperti asam dan alkali, menjadikannya pilihan yang baik untuk produk-produk yang memerlukan ketahanan terhadap korosi. Kedua, PET memiliki sifat tahan terhadap kelembaban dan air, sehingga sering digunakan dalam produk-produk yang terpapar kelembaban tinggi. Ketiga, PET memiliki kekuatan yang baik dan tahan terhadap tekanan, menjadikannya pilihan yang baik untuk produk-produk yang membutuhkan kekuatan mekanis yang tinggi.',
        'kekurangan': 'PET juga memiliki beberapa kekurangan yang perlu dipertimbangkan. Pertama, PET rentan terhadap suhu tinggi dan dapat meleleh atau deformasi jika terpapar suhu yang ekstrem. Kedua, PET memiliki ketahanan rendah terhadap sinar UV dan dapat mengalami degradasi jika terpapar langsung sinar matahari. Ketiga, PET sulit untuk didaur ulang secara efisien dan membutuhkan proses khusus untuk daur ulang yang efektif.',
        'logo': 'https://storage.googleapis.com/trashurebucket/dic_ml/Symbol_Resin_Code_1_PET.svg'
    },
    'PP': {
        'kode': 'pp',
        'nama': 'Polypropylene',
        'manfaat': 'PP (Polypropylene) memiliki beberapa manfaat yang membuatnya populer dalam berbagai aplikasi. Pertama, PP memiliki kekuatan yang tinggi dan tahan terhadap tekanan dan benturan, menjadikannya pilihan yang baik untuk produk-produk yang membutuhkan kekuatan mekanis yang tinggi. Kedua, PP memiliki sifat tahan terhadap bahan kimia, seperti asam dan alkali, sehingga sering digunakan dalam industri kimia. Ketiga, PP memiliki sifat tahan terhadap panas dan memiliki titik leleh yang tinggi, sehingga cocok untuk penggunaan dalam produk-produk yang terekspos suhu tinggi.',
        'kekurangan': 'PP juga memiliki beberapa kekurangan yang perlu dipertimbangkan. Pertama, PP rentan terhadap paparan sinar UV dan dapat mengalami degradasi jika terpapar langsung sinar matahari untuk jangka waktu yang lama. Kedua, PP memiliki ketahanan rendah terhadap suhu rendah, sehingga dapat menjadi rapuh dan mudah retak dalam suhu dingin yang ekstrem. Ketiga, PP sulit untuk didaur ulang secara efisien dan membutuhkan proses khusus untuk daur ulang yang efektif.',
        'logo': 'https://storage.googleapis.com/trashurebucket/dic_ml/Symbol_Resin_Code_5_PP.svg'
    },
    'PS': {
        'kode': 'ps',
        'nama': 'Polystyrene',
        'manfaat': 'PS (Polystyrene) memiliki beberapa manfaat yang membuatnya banyak digunakan dalam industri kemasan dan aplikasi lainnya. Pertama, PS memiliki sifat isolasi termal yang baik, menjadikannya pilihan yang populer untuk wadah makanan dan minuman yang memerlukan perlindungan terhadap perubahan suhu. Kedua, PS adalah bahan yang ringan dan mudah dibentuk, sehingga digunakan dalam berbagai produk seperti gelas, mangkuk, dan wadah makanan sekali pakai. Ketiga, PS memiliki kekuatan yang baik, memberikan perlindungan yang memadai untuk produk yang dikemas.',
        'kekurangan': 'PS juga memiliki beberapa kekurangan yang perlu dipertimbangkan. Pertama, PS sulit untuk didaur ulang secara efektif dan membutuhkan proses daur ulang yang khusus. Hal ini menyebabkan penumpukan limbah PS yang berkontribusi terhadap masalah limbah plastik. Kedua, pembuangan PS yang tidak tepat dapat menyebabkan pencemaran lingkungan dan berdampak negatif pada ekosistem. Ketiga, PS berpotensi untuk melepaskan bahan kimia berbahaya jika terkena suhu tinggi atau kontak dengan bahan kimia tertentu.',
        'logo': 'https://storage.googleapis.com/trashurebucket/dic_ml/Symbol_Resin_Code_6_PS.svg'
    }
}

ort_session = ort.InferenceSession("best.onnx")

img = cv2.imread('pure2jpg.jpg')
img = cv2.resize(img, (640, 640))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = np.transpose(img, (2, 0, 1))
img = np.expand_dims(img, axis=0)
img = img.astype(np.float32) / 255.0

ort_inputs = {ort_session.get_inputs()[0].name: img}
ort_outs = ort_session.run(None, ort_inputs)

max_prob_index = ort_outs[0].argmax()
max_prob_class = class_names[max_prob_index]
full_info = plastic_dict[max_prob_class]
print(full_info)
# for res in ort_outs:
#     max_prob_index = res.probs.data.numpy().argmax()
#     # Map the index to the class name
#     max_prob_class = res.names[max_prob_index]
#     # Print the class with the maximum probability
#     print(f"{max_prob_class}")

{'kode': 'hdpe', 'nama': 'High density polyethylene', 'manfaat': 'manfaat HDPE (High-Density Polyethylene) sebagai bahan plastik termasuk ketahanan terhadap bahan kimia, seperti asam dan alkali, serta tahan terhadap kelembaban dan air. HDPE juga dapat didaur ulang untuk mengurangi limbah plastik dan memiliki kekuatan tarik yang tinggi, membuatnya cocok untuk aplikasi yang membutuhkan kekuatan dan keandalan.', 'kekurangan': 'HDPE juga memiliki beberapa kekurangan. Misalnya, HDPE rentan terhadap suhu tinggi dan dapat deformasi atau meleleh jika terpapar suhu yang ekstrem. Selain itu, HDPE tidak transparan, sehingga tidak cocok untuk produk yang memerlukan transparansi atau tampilan visual. HDPE juga tidak tahan terhadap panas dan dapat retak jika terkena tekanan atau gaya bengkok yang ekstrem.', 'logo': 'https://storage.googleapis.com/trashurebucket/dic_ml/Symbol_Resin_Code_2_HDPE.svg'}


In [6]:
%pip install -r ../requirements.txt

In [82]:
import torch
from ultralytics import YOLO
import cv2
import numpy as np

# Load the YOLOv8 model
model = YOLO("best.pt")

# Class names and plastic information (same as before)
class_names = ["HDPE", "LDPE", "PET", "PP", "PS"]
plastic_dict = {
    "HDPE": {
        "kode": "hdpe",
        "nama": "High density polyethylene",
        "manfaat": "manfaat HDPE (High-Density Polyethylene) sebagai bahan plastik termasuk ketahanan terhadap bahan kimia, seperti asam dan alkali, serta tahan terhadap kelembaban dan air. HDPE juga dapat didaur ulang untuk mengurangi limbah plastik dan memiliki kekuatan tarik yang tinggi, membuatnya cocok untuk aplikasi yang membutuhkan kekuatan dan keandalan.",
        "kekurangan": "HDPE juga memiliki beberapa kekurangan. Misalnya, HDPE rentan terhadap suhu tinggi dan dapat deformasi atau meleleh jika terpapar suhu yang ekstrem. Selain itu, HDPE tidak transparan, sehingga tidak cocok untuk produk yang memerlukan transparansi atau tampilan visual. HDPE juga tidak tahan terhadap panas dan dapat retak jika terkena tekanan atau gaya bengkok yang ekstrem.",
        "logo": "https://upload.wikimedia.org/wikipedia/commons/0/01/Symbol_Resin_Code_2_HDPE.svg",
    },
    "LDPE": {
        "kode": "ldpe",
        "nama": "Low density polyethylene",
        "manfaat": "LDPE (Low-Density Polyethylene) memiliki beberapa manfaat yang membuatnya populer dalam berbagai aplikasi. Pertama, LDPE memiliki sifat tahan terhadap bahan kimia, seperti asam dan alkali, menjadikannya pilihan yang baik untuk produk-produk yang memerlukan ketahanan terhadap korosi. Kedua, LDPE memiliki sifat tahan terhadap kelembaban dan air, sehingga sering digunakan dalam produk-produk yang terpapar kelembaban tinggi. Ketiga, LDPE memiliki kekuatan yang baik dan tahan terhadap tekanan, menjadikannya pilihan yang baik untuk produk-produk yang membutuhkan kekuatan mekanis yang tinggi.",
        "kekurangan": "LDPE juga memiliki beberapa kekurangan yang perlu dipertimbangkan. Pertama, LDPE rentan terhadap suhu tinggi dan dapat meleleh atau deformasi jika terpapar suhu yang ekstrem. Kedua, LDPE memiliki ketahanan rendah terhadap sinar UV dan dapat mengalami degradasi jika terpapar langsung sinar matahari. Ketiga, LDPE sulit untuk didaur ulang secara efisien dan membutuhkan proses khusus untuk daur ulang yang efektif.",
        "logo": "https://upload.wikimedia.org/wikipedia/commons/0/00/Symbol_Resin_Code_4_LDPE.svg",
    },
    "PET": {
        "kode": "pet",
        "nama": "Polyethylene terephthalate",
        "manfaat": "PET (Polyethylene Terephthalate) memiliki beberapa manfaat yang membuatnya populer dalam berbagai aplikasi. Pertama, PET memiliki sifat tahan terhadap bahan kimia, seperti asam dan alkali, menjadikannya pilihan yang baik untuk produk-produk yang memerlukan ketahanan terhadap korosi. Kedua, PET memiliki sifat tahan terhadap kelembaban dan air, sehingga sering digunakan dalam produk-produk yang terpapar kelembaban tinggi. Ketiga, PET memiliki kekuatan yang baik dan tahan terhadap tekanan, menjadikannya pilihan yang baik untuk produk-produk yang membutuhkan kekuatan mekanis yang tinggi.",
        "kekurangan": "PET juga memiliki beberapa kekurangan yang perlu dipertimbangkan. Pertama, PET rentan terhadap suhu tinggi dan dapat meleleh atau deformasi jika terpapar suhu yang ekstrem. Kedua, PET memiliki ketahanan rendah terhadap sinar UV dan dapat mengalami degradasi jika terpapar langsung sinar matahari. Ketiga, PET sulit untuk didaur ulang secara efisien dan membutuhkan proses khusus untuk daur ulang yang efektif.",
        "logo": "https://en.m.wikipedia.org/wiki/File:Symbol_Resin_Code_4_LDPE.svg#/media/File%3APlastic-recyc-01.svg",
    },
    "PP": {
        "kode": "pp",
        "nama": "Polypropylene",
        "manfaat": "PP (Polypropylene) memiliki beberapa manfaat yang membuatnya populer dalam berbagai aplikasi. Pertama, PP memiliki kekuatan yang tinggi dan tahan terhadap tekanan dan benturan, menjadikannya pilihan yang baik untuk produk-produk yang membutuhkan kekuatan mekanis yang tinggi. Kedua, PP memiliki sifat tahan terhadap bahan kimia, seperti asam dan alkali, sehingga sering digunakan dalam industri kimia. Ketiga, PP memiliki sifat tahan terhadap panas dan memiliki titik leleh yang tinggi, sehingga cocok untuk penggunaan dalam produk-produk yang terekspos suhu tinggi.",
        "kekurangan": "PP juga memiliki beberapa kekurangan yang perlu dipertimbangkan. Pertama, PP rentan terhadap paparan sinar UV dan dapat mengalami degradasi jika terpapar langsung sinar matahari untuk jangka waktu yang lama. Kedua, PP memiliki ketahanan rendah terhadap suhu rendah, sehingga dapat menjadi rapuh dan mudah retak dalam suhu dingin yang ekstrem. Ketiga, PP sulit untuk didaur ulang secara efisien dan membutuhkan proses khusus untuk daur ulang yang efektif.",
        "logo": "https://en.m.wikipedia.org/wiki/File:Symbol_Resin_Code_4_LDPE.svg#/media/File%3APlastic-recyc-05.svg",
    },
    "PS": {
        "kode": "ps",
        "nama": "Polystyrene",
        "manfaat": "PS (Polystyrene) memiliki beberapa manfaat yang membuatnya banyak digunakan dalam industri kemasan dan aplikasi lainnya. Pertama, PS memiliki sifat isolasi termal yang baik, menjadikannya pilihan yang populer untuk wadah makanan dan minuman yang memerlukan perlindungan terhadap perubahan suhu. Kedua, PS adalah bahan yang ringan dan mudah dibentuk, sehingga digunakan dalam berbagai produk seperti gelas, mangkuk, dan wadah makanan sekali pakai. Ketiga, PS memiliki kekuatan yang baik, memberikan perlindungan yang memadai untuk produk yang dikemas.",
        "kekurangan": "PS juga memiliki beberapa kekurangan yang perlu dipertimbangkan. Pertama, PS sulit untuk didaur ulang secara efektif dan membutuhkan proses daur ulang yang khusus. Hal ini menyebabkan penumpukan limbah PS yang berkontribusi terhadap masalah limbah plastik. Kedua, pembuangan PS yang tidak tepat dapat menyebabkan pencemaran lingkungan dan berdampak negatif pada ekosistem. Ketiga, PS berpotensi untuk melepaskan bahan kimia berbahaya jika terkena suhu tinggi atau kontak dengan bahan kimia tertentu.",
        "logo": "https://en.m.wikipedia.org/wiki/File:Symbol_Resin_Code_4_LDPE.svg#/media/File%3APlastic-recyc-06.svg",
    },
}  # (Your existing dictionary)

# Load and preprocess the image
img = cv2.imread("plastic60.jpg")  # Load the image
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB

# Perform inference
results = model(img)  # Pass the image to the model

for result in results:
    probs = result.probs  # Probs object for classification outputs
    speed = result.speed  # Speed object for inference time


max_prob_index = probs.data.numpy().argmax()  # Get the index of the class with the highest probability
class_name = result.names[max_prob_index]  # Get the class name
# Get sum of speed
processing_time = speed['preprocess'] + speed['inference'] + speed['postprocess']  # Get the sum of the inference times

confidence = probs.data.numpy()[max_prob_index]
# Mendapatkan waktu pemrosesan

print(class_name, confidence, processing_time,'ms')  # Print the class and its confidence


0: 640x640 OTHER 0.88, LDPE 0.12, PET 0.00, PS 0.00, PP 0.00, 242.8ms
Speed: 24.8ms preprocess, 242.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


TypeError: sum() takes at most 2 arguments (3 given)